In [1]:
%load_ext autoreload
%autoreload 2
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import gseapy as gp
import scanpy as sc

In [3]:
adata = sc.read_h5ad("/home/mmokhtari/INF_analysis/INF_analysis_final.h5ad")

In [4]:
adata.layers['counts'] = adata.X

In [5]:
# preprocessing
#sc.pp.normalize_total(adata, target_sum=1e4)
#sc.pp.log1p(adata)
#adata.layers['lognorm'] = adata.X

In [6]:
# set STIM as class 0, CTRL as class 1, to make categorical
adata.obs['donor'] = pd.Categorical(adata.obs['donor'], categories=["donor1", "donor0"], ordered=True)
indices = adata.obs.sort_values(['Final_Annotation', 'donor']).index
adata = adata[indices,:]

In [14]:
for i in os.listdir("/home/mmokhtari/INF_analysis/DEGs_up/"):
    t = pd.read_csv(("/home/mmokhtari/INF_analysis/DEGs_up/"+i),index_col=0)
# Enricr API
    enr_up = gp.enrichr(t.names,
                    gene_sets='GO_Biological_Process_2021',
                    outdir=None)
# trim (go:...)
    enr_up.res2d.Term = enr_up.res2d.Term.str.split(" \(GO").str[0]
    x= enr_up.res2d
    x=x[x["Adjusted P-value"]<0.05]
    #print(x)
    #x.to_csv("gseapy output/up/"+i)
# dotplot
    #gp.dotplot(enr_up.res2d, figsize=(3,5), title=(i+"Up"), cmap = plt.cm.autumn_r)
    #plt.show()

In [15]:
for i in os.listdir("/home/mmokhtari/INF_analysis/DEGs_down/"):
    t = pd.read_csv(("/home/mmokhtari/INF_analysis/DEGs_down/"+i),index_col=0)
    #print(t)
    # Enricr API
    enr_down = gp.enrichr(t.names,
                    gene_sets='GO_Biological_Process_2021',
                    outdir=None)
    # trim (go:...)
    enr_down.res2d.Term = enr_down.res2d.Term.str.split(" \(GO").str[0]
    x= enr_down.res2d
    #x=x[x["Adjusted P-value"]<0.05]
    #print(x)
    #x.to_csv("gseapy output/down/"+i)
    # dotplot
    #gp.dotplot(enr_down.res2d, figsize=(3,5), title=(i+"Down"), cmap = plt.cm.autumn_r)
    #plt.show()